# Chapter 4. 구조적 API 개요

구조적 API에는 다음과 같은 세 가지 분산 컬렉션 API가 있다.
- Dataset
- DataFrame
- SQL 테이블과 뷰

## 4.1 DataFrame과 Dataset
DataFrame과 Dataset은 잘 정의된 로우와 컬럼을 가지는 분산 테이블 형태의 컬렉션이다. 각 컬럼은 다른 컬럼과 동일한 수의 로우를 가져야 한다(값이 없으면 null 처리). 컬렉션과 로우는 같은 데이터 타입 정보를 가지고 있어야 한다. 이 둘은 결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야 하는지 정의하는 지연 연산의 실행 계획이며 불변성을 가진다. 

## 4.2 스키마
스키마는 데이터프레임의 컬럼명과 데이터 타입을 정의한다. 데이터소스에서 얻거나(스키마 온 리드라고 한다) 직접 정의할 수 있다. 

## 4.3 스파크의 구조적 데이터 타입 개요
스파크는 실행 계획 수립과 처리에 사용하는 자체 데이터 타입 정보를 가지고 있는 카탈리스트 엔진을 사용한다. 스파크 자체 데이터 타입을 지원하는 여러 언어 API와 직접 매핑되며, 각 언어에 대한 매핑 테이블을 가지고 있다. 파이썬을 이용해 구조적 API를 사용하더라도 대부분의 연산은 스파크의 데이터 타입을 사용한다. 

In [2]:
# ex
df = spark.range(500).toDF('number')
df.select(df['number']+10).show()

+-------------+
|(number + 10)|
+-------------+
|           10|
|           11|
|           12|
|           13|
|           14|
|           15|
|           16|
|           17|
|           18|
|           19|
|           20|
|           21|
|           22|
|           23|
|           24|
|           25|
|           26|
|           27|
|           28|
|           29|
+-------------+
only showing top 20 rows



스파크에서 덧셈 연산이 수행되는 이유는 스파크가 지원하는 언어를 이용해 작성된 표현식을 카탈리스트 엔진에서 스파크이 데이터 타입으로 변환해 명령을 처리하기 때문이다. 

### 4.3.1 데이터프레임과 데이터셋 비교
- DataFrame = 비타입형 파이썬에서는 dataset을 dataframe으로 처리해야한다. 
- Dataset = 타입형, 스칼라와 자바만 지원
데이터프레임은 스키마에 명시된 데이터 타입의 일치 여부를 런타임이 되어서야 확인한다. 반면 데이터셋은 스키마에 명시된 데이터 타입의 일치 여부를 컴파일 타임에 확인한다. 
<br>
스파크의 데이터프레임은 row 타입으로 구성된 데이터셋이다. row타입은 스파크가 사용하는 연산에 최적화된 인메모리 포맷의 내부적 표현 방식이다. 이것을 사용하면 가비지 컬렉션과 객체 초기화 부하가 있는 JVM 데이터 타입을 사용하는 대신 자체 데이터 포맷을 사용하기 때문에 효율적인 연산이 가능하다. 

### 4.3.2 컬럼 
- 단순 데이터 타입: 정수형이나 문자열
- 복합 데이터 타입: 배열이나 맵
- null

### 4.3.3 로우
로우는 데이터 레코드. 

## 4.4 구조적 API의 실행과정
1. DataFrame/Dataset/SQL을 이용해 코드를 작성한다.
2. 정상적인 코드라면 스파크가 논리적 실행 계획으로 변환한다.
3. 스파크는 논리적 실행 계획을 물리적 실행 계획으로 변환하며 그 과정에서 추가적인 최적화를 할 수 있는지 확인한다.
4. 스파크는 클러스터에서 물리적 실행 계획(RDD 처리)을 실행한다. 

# Chapter 5 구조적 API 기본 연산
- DataFrame은 레코드와 컬럼으로 구성된다. 
- 스키마는 각 컬럼명과 데이터 타입을 정의한다. 
- 파티셔닝은 DataFrame/Dataset이 클러스터에서 물리적으로 배치되는 형태. 
- 파티셔닝 스키마는 파티션을 배치하는 방법을 정의한다. 

In [3]:
df = spark.read.format('json').load("file:///home/ubuntu/structure api/2015-summary.json")

In [4]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



## 5.1 스키마
스키마는 데이터프레임의 칼럼명과 데이터 타입을 정의한다. 데이터소스에서 스키마를 얻거나 직접 정의할 수 있다.
- 스키마는 여러 개의 structfield 타입 필드로 구성된 structtype 객체이다. 
- structfield는 이름, 데이터 타입, 컬럼이 값이 없거나 Null 일 수 있는지 지정하는 불리언 값을 가진다. 
- 필요한 경우 메타데이터를 지정할 수 있다. 
- 복합 데이터 타입인 structtype을 가질 수 있다. 
- 스파크는 런타임에 데이터 타입이 스키마의 데이터 타입과 일치하지 않으면 오류를 발생시킨다. 

In [5]:
spark.read.format('json').load("file:///home/ubuntu/structure api/2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [6]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

mySchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), True),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
    StructField('count', LongType(), False, metadata = {'hello': 'world'})
])
df = spark.read.format('json').schema(mySchema) \
.load('file:///home/ubuntu/structure api/2015-summary.json')

In [7]:
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

## 5.2 컬럼과 표현식
스파크의 컬럼은 pandas의 dataframe 컬럼과 비슷하다. 표현식으로 DataFrame의 컬럼을 선택, 조작, 제거할 수 있다. 

### 5.2.1 컬럼

In [8]:
# 컬럼은 컬럼명을 카탈로그에 저장된 정보와 비교하기 전까지 미확인 상태로 남는다.
# 분석기가 동작하는 단계에서 컬럼과 테이블을 분석한다. 
# col은 조인 시 유용하다. 
from pyspark.sql.functions import col, column
col('SomeColumnName')
column('SomeColumnName')

Column<b'SomeColumnName'>

### 5.2.2 표현식
표현식은 DataFrame 레코드의 여러 값에 대한 트랜스포메이션 집합을 의미한다. 
- 표현식은 expr 함수로 간단하게 사용할 수 있다. 
- 컬럼은 단지 표현식일 뿐이다.
- 컬럼과 컬럼의 트랜스포메이션은 파싱된 표현식과 동일한 논리적 실행 계획으로 컴파일된다. 

In [9]:
# SQL문에 추가해도 잘 사용할 수 있고 동일한 결과가 나온다. 왜냐하면 실행 시점에 동일한 논리 트리로 컴파일 되기 때문.
from pyspark.sql.functions import expr
expr("(((someCol + 5)*200)-6)<otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

## 5.3 레코드와 로우
- 스파크는 레코드를 row 객체로 표현하고, 값을 생성하기 위해 컬럼 표현식으로 row 객체를 다룬다. 
- row 객체는 내부에 바이트 배열을 가진다. 

In [10]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

### 5.3.1 로우 생성하기
row 객체는 스키마 정보를 가지고 있지 않고, dataFrame만 유일하게 스키마를 가진다. 따라서 row객체를 직접 생성하려면 DataFrame의 스키마와 같은 순서로 값을 명시해야한다. 

In [11]:
from pyspark.sql import Row
myRow = Row('Hello', None, 1, False)

In [12]:
print(myRow[0])
print(myRow[2])

Hello
1


## 5.4 DataFrame의 트랜스포메이션
DataFrame을 다루는 몇 가지 주요 작업
- 로우나 칼럼 추가
- 로우나 칼럼 제거
- 로우를 컬럼으로 변환하거나, 그 반대로 변환
- 컬럼값을 기준으로 로우 순서 변경

### 5.4.1 DataFrame 생성하기

In [13]:
df = spark.read.format('json').load("file:///home/ubuntu/structure api/2015-summary.json")
df.createOrReplaceTempView('dfTable')

In [14]:
# row객체를 가진 seq 타입을 직접 변환해 DataFrame을 생성할 수 있다. 

from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

mySchema = StructType([
    StructField('Some', StringType(), True),
    StructField('col', StringType(), True),
    StructField('names', LongType(), False)
])

myRow = Row('Hello', None, 1)
myDf = spark.createDataFrame([myRow], mySchema)
myDf.show()

+-----+----+-----+
| Some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



사용하기 유용한 methods
- 칼럼이나 표현식을 사용하는 select 메서드
- 문자열 표현식을 사용하는 selectExpr 메서드
- 메서드로 사용할 수 없는 org.apache.spark.sql.functions 패키지에 포함된 다양한 함수

### 5.4.2 select와 selectExpr
이 두개를 사용하면 테이블에 SQL을 실행하는 것처럼 DataFrame에도 실행 가능하다. 

In [15]:
df.select('DEST_COUNTRY_NAME').show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [16]:
# 여러 컬럼 선택
df.select('DEST_COUNTRY_NAME','ORIGIN_COUNTRY_NAME').show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [17]:
from pyspark.sql.functions import expr, col, column
df.select(expr('DEST_COUNTRY_NAME'),
         col('ORIGIN_COUNTRY_NAME'),
         column('DEST_COUNTRY_NAME')) \
.show(2)

+-----------------+-------------------+-----------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-------------------+-----------------+
|    United States|            Romania|    United States|
|    United States|            Croatia|    United States|
+-----------------+-------------------+-----------------+
only showing top 2 rows



In [18]:
# expr을 이용해서 컬럼을 참조할 수 있다.
print(df.select(expr('DEST_COUNTRY_NAME as destination' )).show(2))
print(df.select(expr('DEST_COUNTRY_NAME as destination').alias('DEST_COUNTRY_NAME')).show(2))

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows

None
+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

None


selectExpr 메서드는 새로운 DataFrame을 생성하는 복잡한 표현식을 간단하게 만드는 도구다. 모든 유효한 비집계형 SQL 구문을 지정할 수 있다. 단 칼럼을 식별할 수 있어야한다. 

In [19]:
df.selectExpr('*',
             '(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry').show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [20]:
# 집계함수를 지정할 수 있다.
df.selectExpr('avg(count)', 'count(distinct(DEST_COUNTRY_NAME))').show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



### 5.4.3 스파크 데이터 타입으로 변환하기
가끔은 새로운 컬럼이 아닌 명시적인 값을 스파크에 전달해야되는 경우가 있다. 리터럴이라는 것을 사용하게 된다. 

In [21]:
from pyspark.sql.functions import lit
df.select(expr('*'), lit(1).alias('One')).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



### 5.4.4 컬럼 추가하기

In [22]:
df.withColumn("numberOne",lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [23]:
df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME')).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [24]:
df.withColumn('Destination', expr('DEST_COUNTRY_NAME')).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

### 5.4.5 컬럼명 변경하기 

In [25]:
df.withColumnRenamed('DEST_COUNTRY_NAME','dsst').columns

['dsst', 'ORIGIN_COUNTRY_NAME', 'count']

### 5.4.6 예약 문자와 키워드
예약 문자를 컬럼명에 사용하려면 `을 사용하면 된다. 

In [26]:
dfwithLongColName = df.withColumn('This long column-name',
                                 expr('origin_country_name'))

In [27]:
dfwithLongColName.selectExpr('`this long column-name`',
                            '`this long column-name` as `new col`').show(2)

+---------------------+-------+
|this long column-name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



In [28]:
dfwithLongColName.createOrReplaceTempView('dfTableLong')

In [29]:
# 표현식 대신 문자열을 사용해 명시적으로 컬럼을 참조하면 리터럴로 해석되기 때문에 예약 문자가 포함된 컬럼을 참조할 수 있다.
dfwithLongColName.select(expr('`this long column-name`')).columns

['this long column-name']

### 5.4.7 대소문자 구분
대소문자 구분 따로 안해줘도 된다

### 5.4.8 컬럼 제거하기

In [30]:
df.drop('origin_country_name').columns

['DEST_COUNTRY_NAME', 'count']

In [31]:
dfwithLongColName.drop('origin_country_name','dest_country_name').columns

['count', 'This long column-name']

### 5.4.9 컬럼의 데이터 타입 변경하기


In [33]:
df.withColumn('count2',col('count').cast('string'))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: string]

### 5.4.10 로우 필터링

In [34]:
print(df.filter(col('count') < 2).show(2))
print(df.where('count < 2').show(2))

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

None
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

None


In [36]:
# and 필터를 만들 때는 차례대로 필터를 연결하고 판단은 스파크에게 맡긴다. 
df.where(col('count') < 2).where(col('ORIGIN_COUNTRY_NAME')!= 'Croatia').show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



### 5.4.11 고유한 로우 얻기

In [38]:
print(df.select('origin_country_name', 'dest_country_name').distinct().count())
print(df.select('origin_country_name').distinct().count())

256
125


### 5.4.12 무작위 샘플 만들기
복원 추출이나 비복원 추출의 사용 여부를 지정할 수도 있다.

In [39]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement,fraction,seed).count()

126

### 5.4.13 임의 분할하기
머신러닝할 떄 트레인 테스트 셋으로 나눌 때 유용하다. 임의성을 가지도록 설계됐기 때문에 시드값은 반드시 설정해줘야한다.

In [40]:
dataFrames = df.randomSplit([0.25,0.75],seed)
dataFrames[0].count() > dataFrames[1].count()

False

### 5.4.14 로우 합치기와 추가
데이터프레임은 불변성을 가지고 있기 때문에 레코드 추가를 하는 작업은 불가능하다. 레코드 추가를 하려면 원본과 새로운 데이터프레임과 통합을 해야한다. 


In [41]:
from pyspark.sql import Row


schema = df.schema
newRows = [
    Row("New Country", "Other Country", 5),
    Row("New New Country", "Other Country", 1)
]

parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)

In [42]:
newDF.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New Country|      Other Country|    5|
|  New New Country|      Other Country|    1|
+-----------------+-------------------+-----+



### 5.4.15 로우 정렬하기

In [43]:
df.sort('count').show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [44]:
df.orderBy('count','dest_country_name').show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [45]:
df.orderBy(col('count'),col('dest_country_name')).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [46]:
# 오름차순, 내림차순 정렬도 가능하다. 

from pyspark.sql.functions import desc, asc

print(df.orderBy(expr('count desc')).show(5))
print(df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(5))

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

None
+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+
only showing top 5 rows

None


### 5.4.16 로우 수 제한하기


In [47]:
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [48]:
df.orderBy(expr('count desc')).limit(6).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
+--------------------+-------------------+-----+



### 5.4.17 repartition과 coalesce
자주 필터링하는 컬럼을 기준으로 데이터를 분할하는 것도 최적화 기법이다. 이를 통해 파티셔닝 스키마와 파티션 수를 포함해 클러스터 전반의 물리적 데이터 구성을 제어할 수 있다. 향후에 사용할 파티션 수가 현재 파티션 수보다 많거나 컬럼을 기준으로 파티션을 만드는 경우에만 사용한다.

In [49]:
# repartition 메서드를 호출하면 무조건 전체 데이터를 셔플한다. 
df.rdd.getNumPartitions()

1

In [52]:
df.repartition(5).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Denmark|  152|
|       United States|         Martinique|   43|
|       United States|        Saint Lucia|  136|
|             Ireland|      United States|  335|
|         South Korea|      United States| 1048|
|       United States|              Italy|  438|
|       United States|             Greece|   23|
|Bonaire, Sint Eus...|      United States|   58|
|              France|      United States|  935|
|       United States|             Cyprus|    1|
|       United States|         Montenegro|    1|
|       United States|            Austria|   63|
|           Australia|      United States|  329|
|       United States|        New Zealand|   74|
|       United States|           Suriname|   34|
|       United States|           Malaysia|    3|
|       United States|             Guyana|   63|
|              Taiwa

In [54]:
# 자주 필터링되는 컬럼을 기준으로 파티션을 재분배하는 것이 좋다.
df.repartition(col('dest_country_name')).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|            Anguilla|      United States|   41|
|              Russia|      United States|  176|
|            Paraguay|      United States|   60|
|             Senegal|      United States|   40|
|              Sweden|      United States|  118|
|            Kiribati|      United States|   26|
|              Guyana|      United States|   64|
|         Philippines|      United States|  134|
|            Djibouti|      United States|    1|
|            Malaysia|      United States|    2|
|           Singapore|      United States|    3|
|                Fiji|      United States|   24|
|              Turkey|      United States|  138|
|                Iraq|      United States|    1|
|             Germany|      United States| 1468|
|              Jordan|      United States|   44|
|               Palau|      United States|   30|
|Turks and Caicos ..

In [55]:
# 선택적으로 파티션 수 지정
df.repartition(5, col('DEST_country_name')).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|             Moldova|      United States|    1|
|             Bolivia|      United States|   30|
|             Algeria|      United States|    4|
|Turks and Caicos ...|      United States|  230|
|            Pakistan|      United States|   12|
|    Marshall Islands|      United States|   42|
|            Suriname|      United States|    1|
|              Panama|      United States|  510|
|         New Zealand|      United States|  111|
|             Liberia|      United States|    2|
|             Ireland|      United States|  335|
|              Zambia|      United States|    1|
|            Malaysia|      United States|    2|
|               Japan|      United States| 1548|
|    French Polynesia|      United States|   43|
|           Singapore|      United States|    3|
|             Denmark|      United States|  153|
|               Spai

coalesce는 전체 데이터를 셔플하지 않고 파티션을 병합하려는 경우에 사용한다. 

In [57]:
df.repartition(5, col('DEST_country_name')).coalesce(2).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|             Moldova|      United States|    1|
|             Bolivia|      United States|   30|
|             Algeria|      United States|    4|
|Turks and Caicos ...|      United States|  230|
|            Pakistan|      United States|   12|
|    Marshall Islands|      United States|   42|
|            Suriname|      United States|    1|
|              Panama|      United States|  510|
|         New Zealand|      United States|  111|
|             Liberia|      United States|    2|
|             Ireland|      United States|  335|
|              Zambia|      United States|    1|
|            Malaysia|      United States|    2|
|               Japan|      United States| 1548|
|    French Polynesia|      United States|   43|
|           Singapore|      United States|    3|
|             Denmark|      United States|  153|
|               Spai